In [1]:
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.svm import LinearSVC
from sklearn.kernel_approximation import PolynomialCountSketch
from sklearn.pipeline import Pipeline, make_pipeline
import time
import numpy as np # linear algebra
import pandas as pd # data processing, 
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
%matplotlib inline


import os
from pathlib import Path
import time
from collections import OrderedDict, namedtuple

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=87f658d3cf02ebce325dc3637503adb378bff949a12a7413fc85b1c6b2887d05
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [3]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [4]:
if IN_COLAB:

    from google.colab import drive
    drive._mount('/drive', force_remount=True)
    datadir = '/drive/My Drive/Colab Notebooks/Datasets/'
    optim_dir = datadir + '/optim_dir/'
    results_folder = datadir+'/results/UCI/'
    Path(results_folder).mkdir(parents=True, exist_ok=True)
    Path(optim_dir).mkdir(parents=True, exist_ok=True)
else:
  results_folder = 'results'
  optim_dir = 'optim_dir'
  Path("results").mkdir(parents=True, exist_ok=True)
  Path(optim_dir).mkdir(parents=True, exist_ok=True)


# from opfunu.cec_basic.cec2014_nobias import *
# from mealpy.swarm_based.PSO import BasePSO

Mounted at /drive


In [5]:
import sys

!pip install category_encoders
import category_encoders as ce

if IN_COLAB:
  
  scripts_dir = '/drive/My Drive/Colab Notebooks/scripts/'
  sys.path.insert(1, scripts_dir)
# from opfunu.cec_basic.cec2014_nobias import *
# from mealpy.swarm_based.PSO import BasePSO

# insert at 1, 0 is the script path (or '' in REPL)
else:
    sys.path.insert(1, 'scripts')
from bayes_opt import BayesianOptimization
import pickle

     |████████████████████████████████| 82 kB 362 kB/s 


In [6]:
# Enable GPU processing
# if IN_COLAB:
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# else:
#     device = torch.device("cpu")
# print(f'Device type: {device}')

In [7]:
# Set random seed
seed = 777
# torch.manual_seed(seed)
model = None

In [8]:
import random
def seeding(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    os.environ['TF_CUDNN_DETERMINISTIC'] = str(SEED)
    # tf.random.set_seed(SEED)
    print('seeding done!!!')
seeding(seed)

seeding done!!!


In [9]:
X, y = fetch_covtype(return_X_y=True)

In [10]:
X

array([[2.596e+03, 5.100e+01, 3.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.590e+03, 5.600e+01, 2.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.804e+03, 1.390e+02, 9.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [2.386e+03, 1.590e+02, 1.700e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.384e+03, 1.700e+02, 1.500e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.383e+03, 1.650e+02, 1.300e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.2, test_size=0.8, random_state=0
)

In [12]:
X_train.shape, X_test.shape # Get the dimensions of features

((116202, 54), (464810, 54))

In [13]:
# Create a class to manage the training / hyperparameter runs
class RunManager():
  def __init__(self):
    self.run_params = None
    self.run_count = 0
    self.run_data = []
    
    #---
    self.results = None

  def begin_run(self, run):
    self.run_params = run
    self.run_count += 1
    

  def end_run(self, train_accuracy, test_accuracy):
    # self.epoch_count = 0
    # train_accuracy = 
    # test_accuracy = 

    print(f'run_params: {self.run_params} Train Acc: {train_accuracy} Test Acc: {test_accuracy}')

    results = OrderedDict()
    results['run'] = self.run_count
    results['train acc'] = train_accuracy
    results['test acc'] = test_accuracy
    
    # ---
    self.results = results

    for k, v in self.run_params.items():
      results[k] = v

    self.run_data.append(results)

  def save_output(self, filename):
    if filename:
      filename = filename
      pd.DataFrame.from_dict(self.run_data, orient='columns').to_csv(f'{filename}.csv')
      
      # with open(f'{filename}.json', 'w', encoding='utf-8') as f:
      #   json.dump(self.run_data, f, ensure_ascii=False, indent=4)

      # print('Results saved to disk')

    return pd.DataFrame.from_dict(self.run_data, orient='columns')


In [14]:
# Define training loop execution
def execution_loop(m, args): # args is given by the optimizer
    agent = {}
    agents = []
    
    agent['n'] = args[0].astype(int)
    agent['m'] = int(args[1])
    agents.append(agent)
#     m = RunManager()
    for run in agents: # this should be one. a particle in pso

        # print(f'Run Params: {run}')

        m.begin_run(run)

        num_predictors = X_train.shape[1]
        if run['m'] <= num_predictors:

          rfc = RandomForestClassifier(n_estimators=run['n'], max_features=run['m'], random_state=42)
          rfc.fit(X_train, y_train)

          y_pred_train = rfc.predict(X_train)
          y_pred_test = rfc.predict(X_test)

          train_score = accuracy_score(y_train, y_pred_train)
          test_score = accuracy_score(y_test, y_pred_test)
        else:
          train_score = 0
          test_score = 0

        m.end_run(train_score, test_score)
    return m.results['test acc']

In [15]:
# Define training run hyperparameters
params = {
    'n' : [1, 200],
    'm' : [1, 20]
}

# params = OrderedDict(
#     lr = [0, 1],
#     batch_size = [20 2000],
#     n_epochs = [3]
# )

In [16]:
m = RunManager()
timestr = time.strftime("%Y%m%d-%H%M%S")
filename = f'Run_Results-UCI-CovType-BO-{timestr}'
# filename = 'Run_Results-PSO-20210504-035642-cont.csv' # this is for trying to continue from last optim save


In [17]:
def run_train_model(agent):
    test_acc = execution_loop(m, agent)
    m.save_output(f'{results_folder}/{filename}')
    print("test acc: ", test_acc)
    return test_acc * -1

In [18]:
def run_train_model_proxy(n_estimators, max_features):
    agent = [n_estimators, max_features]
    print(agent)
    return run_train_model(agent) * -1

In [19]:
verbose = True
pop_size = 15   
max_iter = 10

obj_func = run_train_model_proxy

lb = [params['n'][0], params['m'][0]]
ub = [params['n'][1], params['m'][1]]

tic = time.perf_counter()
# Run Bayesian Optimization

params_gbm ={
    'max_features':(1, 20), 'n_estimators':(1, 200)
}

gbm_bo = BayesianOptimization(f = obj_func, pbounds = params_gbm, random_state=777)
gbm_bo.maximize(init_points=15, n_iter=135)

toc = time.perf_counter()
print(f'Time to execute: {toc - tic:0.4f} seconds')
# print(f'Best Position: {[int(x) for x in best_pos1]} Best Solution: {best_fit1:0.5f}')
print(gbm_bo.max)


|   iter    |  target   | max_fe... | n_esti... |
-------------------------------------------------
[61.16896533179672, 3.9006109631251182]
run_params: {'n': 61, 'm': 3} Train Acc: 0.9999483657768369 Test Acc: 0.9011639164389751
test acc:  0.9011639164389751
|  1        |  0.9012   |  3.901    |  61.17    |
[92.5122079801848, 2.178691879576678]
run_params: {'n': 92, 'm': 2} Train Acc: 1.0 Test Acc: 0.8995976850756223
test acc:  0.8995976850756223
|  2        |  0.8996   |  2.179    |  92.51    |
[185.47241250222905, 16.869814304314936]
run_params: {'n': 185, 'm': 16} Train Acc: 1.0 Test Acc: 0.9294464404810567
test acc:  0.9294464404810567
|  3        |  0.9294   |  16.87    |  185.5    |
[153.93074816220832, 14.812790702062332]
run_params: {'n': 153, 'm': 14} Train Acc: 1.0 Test Acc: 0.9274778941933263
test acc:  0.9274778941933263
|  4        |  0.9275   |  14.81    |  153.9    |
[129.16182912212687, 6.114896262971671]
run_params: {'n': 129, 'm': 6} Train Acc: 1.0 Test Acc: 0.9136894